# About: 設定ファイルの変更--httpd.conf

---

リバースプロキシコンテナの `httpd.conf` の設定内容を変更します。

## 概要

設定ファイルを変更する例としてリバースプロキシコンテナの`httpd.conf`を編集して、起動時のサーバプロセス数を変更してみます。

![処理の流れ](images/moodle-032-01.png)

操作手順は以下のようになります。

1. VCノードに配置されている設定ファイルをNotebook環境に取得する
2. 取得したファイルのバックアップを作成する
3. Notebookの編集機能を利用して設定ファイルの変更をおこなう
4. 変更した設定ファイルをVCノードに配置する
5. 設定ファイルの変更を反映させるためにコンテナを再起動する

アプリケーションテンプレートによって構築したMoodle環境では、利用者が変更する可能性のある `httpd.conf` などの設定ファイルをVCノードに配置しています。VCノードに配置したファイルはコンテナ内から参照するために[bind mount](https://docs.docker.com/storage/bind-mounts/)の設定を行っています。このため、設定ファイルを編集する場合はコンテナ内のパス名とVCノードのパス名のどちらを指定するか、またbindする際のマウントポイントがどこになっているかなどについて留意する必要があります。

リバースプロキシコンテナにおける設定ファイルの対応関係を以下の表に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ内のパス</th>
    <th style="text-align:left;">VCノードのパス</th>
    <th style="text-align:left;">コンテナ側のマウントポイント</th>
    <th style="text-align:left;">VCノード側のマウントポイント</th>
  </tr>
  <tr>
    <td style="text-align:left;">/usr/local/apache2/conf/httpd.conf</td>
    <td style="text-align:left;">/opt/moodle/proxy/conf/httpd.conf</td>
    <td style="text-align:left;">/usr/local/apache2/conf/</td>
    <td style="text-align:left;">/opt/moodle/proxy/conf/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/usr/local/apache2/conf/moodle-proxy.conf</td>
    <td style="text-align:left;">/opt/moodle/proxy/conf/moodle-proxy.conf</td>
    <td style="text-align:left;">/usr/local/apache2/conf/</td>
    <td style="text-align:left;">/opt/moodle/proxy/conf/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/usr/local/apache2/conf/extra/</td>
    <td style="text-align:left;">/opt/moodle/proxy/conf/extra/</td>
    <td style="text-align:left;">/usr/local/apache2/conf/</td>
    <td style="text-align:left;">/opt/moodle/proxy/conf/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/usr/local/apache2/conf/cert/server.crt</td>
    <td style="text-align:left;">/opt/moodle/cert/server.crt</td>
    <td style="text-align:left;">/usr/local/apache2/conf/cert/</td>
    <td style="text-align:left;">/opt/moodle/cert/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/usr/local/apache2/conf/cert/server.key</td>
    <td style="text-align:left;">/opt/moodle/cert/server.key</td>
    <td style="text-align:left;">/usr/local/apache2/conf/cert/</td>
    <td style="text-align:left;">/opt/moodle/cert/</td>
  </tr>
</table>

## パラメータの設定

変更対象のコンテナ名、設定ファイル名などを指定します。

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### チェック

指定された `ugroup_name` の値が適切なものかチェックします。

`ugroup_name` に対応する `group_vars` ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (ugroup_name + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {ugroup_name + '.yml'}")

`ugroup_name`に対応する VCノードが実行中であり Ansible で操作できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping

### コンテナの指定

設定ファイルを変更する対象となるコンテナを指定します。

現在実行中のコンテナの一覧を確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle docker-compose ps --services'

表示されたコンテナ一覧から、対象とするコンテナ名を指定してください。

In [ ]:
target_container = 'proxy'

### 設定ファイルの指定

変更する設定ファイルのパスを指定してください。ここではコンテナ内のパスを指定してください。

In [ ]:
target_file = '/usr/local/apache2/conf/httpd.conf'

VCノードとコンテナとで bind mountする際に基準となるコンテナ側のマウントポイントを指定してください。

In [ ]:
relative_to = '/usr/local/apache2/conf'

## 設定ファイルの編集

リバースプロキシコンテナの設定ファイル`httpd.conf`をローカル環境に取得しJupyter Notebookの編集機能を用いて設定ファイルを編集します。

![設定ファイルの取得、編集](images/moodle-032-03.png)

次のセルを実行すると、以下の手順が実行されます。

1. VCノードに配置されているApache HTTP Serverコンテナの設定ファイル(`httpd.conf`)をローカル環境に取得する
2. 取得した設定ファイルのバックアップを作成する
3. Jupyter Notebookの編集機能を利用して設定ファイルを編集するためのリンクを表示する

In [ ]:
%run scripts/edit_conf.py
fetch_conf(ugroup_name, target_container, target_file, relative_to)

上のセルの出力に表示されているリンクをクリックして設定ファイルの編集を行ってください。

この例では、ファイルの末尾に以下の内容を追加してください。

```
<IfModule mpm_event_module>
  ServerLimit              16
  StartServers             16
  MinSpareThreads         400
  MaxSpareThreads         400
  ThreadsPerChild          25
</IfModule>
```

この設定を追加することによりApache HTTP Serverの起動時のプロセス数がデフォルトの 3 から 16 に増えます。

> ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

ローカル環境に取得したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/httpd.conf`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

また、バックアップファイルは以下のパスに格納されます。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/httpd.conf.orig`

次のセルを実行すると編集の前後での差分を確認することができます。

In [ ]:
show_local_conf_diff(ugroup_name, target_container, target_file)

## 編集した設定ファイルの反映

編集したファイルをVCノードに配置して、設定ファイルの変更内容をコンテナに反映させます。

![設定ファイルの反映](images/moodle-032-04.png)

### 変更前の状態を確認する

設定ファイル変更の反映前の状態を確認します。

Apache HTTP Serverのプロセス数に関する設定を変更したので、Apache HTTP Serverコンテナのプロセス一覧を確認します。

Apache HTTP Serverコンテナには `ps` コマンドがインストールされていないので、`procps`パッケージを追加します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle docker-compose exec -T {target_container} \
    bash -c "apt-get update && apt-get install -y procps"'

プロセス一覧を表示します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle docker-compose exec -T {target_container} ps ax '

### 編集内容の反映

前章で編集した設定ファイル `httpd.conf` を Apache HTTP Serverコンテナに反映します。

次のセルを実行すると、以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイル(`httpd.conf`)をVCノードに配置する
3. コンテナを再起動して変更した設定ファイルの反映を行う

In [ ]:
apply_conf(ugroup_name, target_container, target_file, relative_to)

### 変更後の状態を確認する

設定ファイル変更の反映後の状態を確認します。

Apache HTTP Serverコンテナのプロセス一覧を確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle docker-compose exec -T {target_container} ps ax '

## 変更を取り消す

編集前の設定ファイルの状態に戻します。

![設定ファイル変更の取り消し](images/moodle-032-05.png)

次のセルを実行すると、以下の手順が実行されます。

1. 編集後と編集前の設定ファイルの差分を表示する
2. 編集前の設定ファイルをVCノードに配置する
3. コンテナを再起動して設定値の反映を行う

In [ ]:
revert_conf(ugroup_name, target_container, target_file, relative_to)

設定ファイルを戻した後の状態を確認します。Apache HTTP Serverコンテナのプロセス一覧を確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle docker-compose exec -T {target_container} ps ax '